In [1]:
from datasets import load_dataset, Value, Sequence, Features

data_files={"train": r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face\sample_data\train.json",
           "validation": r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face\sample_data\val.json",
           "test": r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face\sample_data\test.json"}
        
#data_files={"train": r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face\sample_data\train.json"}


#dataset = load_dataset("json", data_files=data_files)

In [2]:
dataset = load_dataset("json", data_files=data_files)

Using custom data configuration default-0eba7e56df251bfe
Found cached dataset json (C:/Users/jimmc/.cache/huggingface/datasets/json/default-0eba7e56df251bfe/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [4]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",     #only truncates the context nothing else
        return_offsets_mapping=True,  # map the start and end positions of the answer to the original context 
        padding="max_length",
    )
    
    offset_mapping = inputs.pop("offset_mapping")
    #print(offset_mapping)
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    #print(answers)
    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        #start_char = answer["answer_start"][0]
        #end_char = answer["answer_start"][0] + len(answer["text"][0])  #start char + length giver chars with answer
        start_char = answer[0]["answer_start"]
        end_char = answer[0]["answer_start"] + len(answer[0]["text"])  
        sequence_ids = inputs.sequence_ids(i)  #no clue what this does
        #print(answer, end_char, sequence_ids)
        # Find the start and end of the context        ; not sure how this section works
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
    
        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
            
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
            
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [5]:
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

  0%|          | 0/13 [00:00<?, ?ba/s]

Loading cached processed dataset at C:/Users/jimmc/.cache/huggingface/datasets/json/default-0eba7e56df251bfe/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab\cache-bdbb438f341af283.arrow
Loading cached processed dataset at C:/Users/jimmc/.cache/huggingface/datasets/json/default-0eba7e56df251bfe/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab\cache-02447fd093a73f65.arrow


In [6]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [7]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForQuestionAnswering: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

In [1]:
tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["validation"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

NameError: name 'model' is not defined

In [12]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 16
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_set) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [13]:
import tensorflow as tf

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

In [14]:
model.fit(tf_train_set, validation_data=tf_validation_set, epochs=3)

Epoch 1/3
787/787 [==============================] - 22391s 28s/step - loss: 4.0266 - end_logits_loss: 1.9827 - start_logits_loss: 2.0439 - end_logits_accuracy: 0.4912 - start_logits_accuracy: 0.4697 - val_loss: 3.0484 - val_end_logits_loss: 1.4668 - val_start_logits_loss: 1.5816 - val_end_logits_accuracy: 0.6055 - val_start_logits_accuracy: 0.5682
Epoch 2/3
787/787 [==============================] - 22095s 28s/step - loss: 1.9746 - end_logits_loss: 0.9792 - start_logits_loss: 0.9954 - end_logits_accuracy: 0.7141 - start_logits_accuracy: 0.7000 - val_loss: 3.0012 - val_end_logits_loss: 1.4173 - val_start_logits_loss: 1.5839 - val_end_logits_accuracy: 0.6292 - val_start_logits_accuracy: 0.5800
Epoch 3/3
787/787 [==============================] - 22121s 28s/step - loss: 1.1539 - end_logits_loss: 0.5758 - start_logits_loss: 0.5781 - end_logits_accuracy: 0.8203 - start_logits_accuracy: 0.8124 - val_loss: 3.3549 - val_end_logits_loss: 1.5727 - val_start_logits_loss: 1.7822 - val_end_logits_

In [15]:
model.save_pretrained(r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face\V2")
tokenizer.save_pretrained(r"C:\Users\jimmc\Documents\UCSD_Bootcamp\Hugging_Face\V2")

('C:\\Users\\jimmc\\Documents\\UCSD_Bootcamp\\Hugging_Face\\V2\\tokenizer_config.json',
 'C:\\Users\\jimmc\\Documents\\UCSD_Bootcamp\\Hugging_Face\\V2\\special_tokens_map.json',
 'C:\\Users\\jimmc\\Documents\\UCSD_Bootcamp\\Hugging_Face\\V2\\vocab.txt',
 'C:\\Users\\jimmc\\Documents\\UCSD_Bootcamp\\Hugging_Face\\V2\\added_tokens.json',
 'C:\\Users\\jimmc\\Documents\\UCSD_Bootcamp\\Hugging_Face\\V2\\tokenizer.json')

In [16]:
from transformers import pipeline
question_answerer = pipeline(task="question-answering", model=model, tokenizer=tokenizer)

In [17]:
exact_answers = []
contains_answer = []
wrong = []
scores = []
for text_row in dataset['test']:
    result = question_answerer(question = text_row['question'], context = text_row['context'])
    correct_start = text_row['answers'][0]['answer_start'] 
    correct_end = text_row['answers'][0]['answer_start'] + len(text_row['answers'][0]['text'])
    scores.append(result['score'])
   
    if (correct_start == result['start']) and (correct_end == result['end']):
        exact_answers.append(1)
        contains_answer.append(1)
        wrong.append(0)
   
    
    elif (correct_start != result['start']) or (correct_end != result['end']):
        
        if (correct_start >= result['start']) and (correct_end <= result['end']):
            exact_answers.append(0)
            contains_answer.append(1)
            wrong.append(0)
        else:
            exact_answers.append(0)
            contains_answer.append(0)
            wrong.append(1)
    else:
        pass

In [18]:
Accuracy = sum(exact_answers)/len(dataset['test'])
Accuracy

0.5004612546125461

In [19]:
sum(contains_answer)/len(dataset['test'])

0.6164667896678967

In [20]:
import numpy as np

In [21]:
np_scores = np.array(scores)
np_exact = np.array(exact_answers)
np_contains = np.array(contains_answer)
np_wrong = np.array(wrong)

In [22]:
#np.sum(np_exact[np_scores >= .5])
#len(np_exact[np_scores >= .5])
TPE = np.sum(np_exact[np_scores >= .5])
TPE
FPE = len(np_exact[np_scores >= .5]) - TPE
FPE
Exact_Percison = TPE/(TPE+FPE)
Exact_Percison

0.661923076923077

In [23]:
TPC = np.sum(np_contains[np_scores >= .5])
TPC
FPC = len(np_contains[np_scores >= .5]) - TPC
FPC
Containing_Percison = TPC/(TPC+FPC)
Containing_Percison

0.7461538461538462

In [24]:
FNE = np.sum(np_exact[np_scores < .5])
FNE
TNE = len(np_exact[np_scores < .5]) - FNE
TNE
Exact_Recall = TPE/(TPE + FNE)
Exact_Recall

0.7930875576036867

In [25]:
FNC = np.sum(np_exact[np_scores < .5])
FNC
TNC = len(np_exact[np_scores < .5]) - FNC
TNC
Containing_Recall = TPC/(TPC + FNC)
Containing_Recall

0.8120552532440352